In [1]:
import sys
#sys.path.append("/home/dictus/dmt")
from dmt.analysis import StructuredAnalysis

In [2]:
from neuro_dmt.library.users.hugo.models.bluebrain import\
    CircuitModel, CircuitAdapter

('layer', 'region', 'mesocolumn', 'height', 'depth')
('layer', 'region', 'mesocolumn', 'height', 'depth', 'morphology_subclass', 'synapse_class', 'mtype')
('layer', 'region', 'mesocolumn', 'height', 'depth')
('layer', 'region', 'mesocolumn', 'height', 'depth')
('region', 'layer', 'mesocolumn', 'height', 'depth', 'mtype', 'synapse_class', 'morphology_class', 'morphology_subclass')
('presynaptic', 'postsynaptic')
('presynaptic', 'postsynaptic')
('presynaptic', 'postsynaptic')
('presynaptic', 'postsynaptic')
('presynaptic', 'postsynaptic')
('region', 'layer', 'mesocolumn', 'hypercolumn', 'height', 'depth', 'mtype', 'morphology_subclass', 'synapse_class', 'morphology_class')
('presynaptic', 'postsynaptic')


In [3]:
from dmt.tk.plotting.heat import HeatPlot
from dmt.tk.plotting.circle import CirclePlot
from neuro_dmt import terminology
import pandas as pd
import numpy as np

# TODO: rename value_callback to label_callback
# TODO: callback should not recieve 'pre'
def custom_label_callback(row):
    return row[terminology.cell.mtype]

import matplotlib
custom_cmap = matplotlib.colors.ListedColormap([
    [129/255, 83/255, 120/255, 1.0],
    [220/255, 224/255, 34/255, 1.0],
    [91/255, 165/255, 217/255, 1.0],
    [255/255, 35/255, 40/255, 1.0],
    [140/255, 208/255, 105/255, 1.0]])

def hue_by_layer_shade_by_sclass(groups):
    layers = []
    for g in groups.values():
        if g[terminology.circuit.layer] not in layers:
            layers.append(g[terminology.circuit.layer])

    maxl = len(layers) - 1 if len(layers) > 1 else 1
    hues = {label: custom_cmap(layers.index(g[terminology.circuit.layer]) / maxl) 
            for label, g in groups.items()}
    
    def set_shade(hue, sclass):
        print(hue, sclass)
        if 'EXC' in sclass:
            return hue
        else:
            return hue * np.array([0.5, 0.5, 0.5, 1.0])
        
    colors = {label: {'edgecolor': (0.0, 0.0, 0.0, 1.0),
                    'facecolor': set_shade(hues[label],
                                           group[terminology.cell.synapse_class])}
            for label, group in groups.items()}
    return colors

circle_plot = CirclePlot(space_between=0.01,
                         color_callback=hue_by_layer_shade_by_sclass,
                         min_conn_size=0.002, label_callback=custom_label_callback,
                         connection_alpha=0.8)


In [4]:
from collections import OrderedDict
def mtype_to_mtype_pathways(adapter, model):
    """
    gets the available mtypes from the model
    effectively makes this a 'by mtype' validation
    """
    mtypes = adapter.get_mtypes(model,
                                properties=[
                                    terminology.cell.mtype,
                                    terminology.circuit.layer, 
                                    terminology.cell.synapse_class])
    def list_if_not_list(item):
        if isinstance(item, list):
            return item
        return [item]
    sorted_mtypes = sorted(mtypes,
                           key=lambda d: (
                               list_if_not_list(d[terminology.circuit.layer]),
                               d[terminology.cell.synapse_class]))
    out= [OrderedDict([(terminology.cell.presynaptic, pre),
                       (terminology.cell.postsynaptic, post)])
          for post in sorted_mtypes for pre in sorted_mtypes]
    return out


In [5]:
from dmt.model.interface import Interface
from dmt.tk.terminology import Term

In [12]:
# TODO: parralelize measuements?
# TODO: why does AdapterInterface need to be a subclass of Interface?
class ByMtypeSynapsesPerConnectionInterface(Interface):
    
    __measurement__ = "synapses_per_connection"
    @terminology.use(
        terminology.cell.presynaptic,
        terminology.cell.postsynaptic)
    def get_synapses_per_connection(self, model, **parameters):
        """
        get the number of synapses per connection from the model
        for the pathway defined be pre and post mtype
            
        Arguments:
            model: the model to get the data from
            {parameters}
            
        Returns:
            Array: number of synapses for each connection sampled
            """
        raise NotImplementedError
        
    def get_mtypes(self, model,
                   properties=(terminology.cell.mtype,
                               terminology.circuit.layer,
                               terminology.cell.synapse_class)):
        """
        get all the mtypes represented in the model
        potentially with data on the layer(s) and sclass of each mtype
        """
        raise NotImplementedError

        
def measure_syn_per_conn(adapter, model, **parameters):
    return adapter.get_synapses_per_connection(model, **parameters)
measure_syn_per_conn.phenomenon = Term('Synapses per Connection', 
                                       "the mean number of synapses on each connection between cells")
        
ByMtypeSynapsesPerConnectionAnalysis = StructuredAnalysis(
    measurement_parameters=mtype_to_mtype_pathways,
    AdapterInterface=ByMtypeSynapsesPerConnectionInterface,
    sample_measurement=measure_syn_per_conn,
    plotter=HeatPlot(label_callback=custom_label_callback))

('presynaptic', 'postsynaptic')


In [13]:
# TODO: decorate AdapterInterface method so it behaves as measurement_method
#       provide phenomenon as decorator argument
class ByMtypeConnectionProbabilityInterface(Interface):
    
    __measurement__ = "connection_probability"
    @terminology.use(
        terminology.cell.presynaptic,
        terminology.cell.postsynaptic)
    def get_connection_probability(self, model, **parameters):
        """
            get the probability of a connection between cells
            
            Arguments:
                model: the adapted model to extract data from
                {parameters}
            
            Returns:
                Array: number of synapses for each connection sampled
        """
        raise NotImplementedError
        
    def get_mtypes(self, model,
                   properties=(terminology.cell.mtype,
                               terminology.circuit.layer,
                               terminology.cell.synapse_class)):
        """
        get all the mtypes represented in the model
        potentially with data on the layer(s) and sclass of each mtype
        """
        raise NotImplementedError
        

def measure_connection_probability(adapter, model, **parameters):
    connprob = adapter.get_connection_probability(model, **parameters)
    return connprob if connprob != 0 else np.nan
measure_connection_probability.phenomenon =\
    Term('Connection Probability',
         "The probability of a connection existing between two random cells, "
         "one from a presynaptic group, one from a postsynaptic group")
        
ByMtypeConnectionProbabilityAnalysis = StructuredAnalysis(
    measurement_parameters=mtype_to_mtype_pathways,
    AdapterInterface=ByMtypeConnectionProbabilityInterface,
    sample_measurement=measure_connection_probability,
    plotter=HeatPlot(label_callback=custom_label_callback))

('presynaptic', 'postsynaptic')


In [14]:

class ByMtypePathwaySynapsesInterface(Interface):
    
    @terminology.use(
        terminology.cell.presynaptic,
        terminology.cell.postsynaptic)
    def get_pathway_synapses(self, model, **parameters):
        """
        get the total number of synapses on one group of cells from another
            
        Arguments:
            model: the adapted model to extract data from
            {parameters}
            
        Returns:
            Array: number of synapses for each connection sampled
        """
        raise NotImplementedError

    def get_mtypes(self, model,
                   properties=(
                       terminology.cell.mtype,
                       terminology.circuit.layer, 
                       terminology.cell.synapse_class)):
        """
        get all the mtypes represented in the model
        potentially with data on the layer(s) and sclass of each mtype
        """
        raise NotImplementedError    

        
def measure_pathway_synapses(adapter, model, **parameters):
    return adapter.get_pathway_synapses(model, **parameters)
measure_pathway_synapses.phenomenon =\
    Term("Pathway Synapses",
         "The total number of synapses from one cell group to another")
        
ByMtypePathwaySynapsesAnalysis = StructuredAnalysis(
    measurement_parameters=mtype_to_mtype_pathways,
    AdapterInterface=ByMtypeConnectionProbabilityInterface,
    sample_measurement=measure_pathway_synapses,
    default_adapter=CircuitAdapter(),
    plotter=circle_plot)


('presynaptic', 'postsynaptic')


In [15]:
import numpy as np

class DummyAdapter:
    
    label = 'dummy model'
    
    def get_synapses_per_connection(self, model, **params):
        return np.mean(np.random.poisson(np.random.poisson(10), 1 + np.random.poisson(1)))
    
    def get_pathway_synapses(self, model, **params):
        return np.mean(np.random.poisson(np.random.poisson(10), 1 + np.random.poisson(1)))
    
    def get_connection_probability(self, model, **params):
        return np.mean(np.random.poisson(np.random.poisson(10), 1 + np.random.poisson(1)))
    
    def get_mtypes(self, model, properties=()):
        return [
            {terminology.cell.mtype: letter,
             terminology.circuit.layer: [1], 
             terminology.cell.synapse_class: 'EXC'} 
            for letter in ['aaaaaaaa', 'bbbbbbbb', 'cccccccc', 'dddddd', 'eeeeeeee', 'fffffff']]

In [16]:
ByMtypeConnectionProbabilityAnalysis((DummyAdapter(), ''))


100%|██████████| 720/720 [00:00<00:00, 30725.63it/s]


In [17]:
ByMtypeSynapsesPerConnectionAnalysis((DummyAdapter(), ''))

100%|██████████| 720/720 [00:00<00:00, 34803.09it/s]


In [18]:
ByMtypePathwaySynapsesAnalysis((DummyAdapter(), ''))

100%|██████████| 720/720 [00:00<00:00, 35278.37it/s]
/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/dmt/analysis/structured.py:327: Warning: "
            Plotting failed:
            type object 'data' has no attribute 'all',
            Traceback (most recent call last):
  File "/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/dmt/analysis/structured.py", line 319, in _get_report
    fig = self.plotter(measurements, phenomenon=self.phenomenon)
  File "/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/dmt/tk/plotting/__init__.py", line 166, in __call__
    for label, dataframe in dataframes_dict.items()]
  File "/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/dmt/tk/plotting/__init__.py", line 166, in <listcomp>
    for label, dataframe in dataframes_dict.items()]
  File "/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/dmt/tk/plotting/circle.py", line 411, in plot
    = self.__plot_components_

In [19]:
thalamus_circuit =\
    CircuitModel(
    "/gpfs/bbp.cscs.ch/project/proj55/iavarone/releases/circuits/O1/2019-07-18/CircuitConfig")

FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/bbp.cscs.ch/project/proj55/iavarone/releases/circuits/O1/2019-07-18/CircuitConfig'

In [ ]:
thalamus_pathway_synapses = ByMtypePathwaySynapsesAnalysis((CircuitAdapter(), thalamus_circuit))

In [ ]:
thalamus_connprob = ByMtypeConnectionProbabilityAnalysis((CircuitAdapter(), thalamus_circuit))

In [ ]:
thalamus_synperconn = ByMtypeSynapsesPerConnectionAnalysis((CircuitAdapter(), thalamus_circuit))

In [ ]:
thalamus_pathway_synapses.measurement


In [ ]:
proj1_circuit = "/gpfs/bbp.cscs.ch/project/proj1/circuits/SomatosensoryCxS1-v5.r0/O1/merged_circuit/CircuitConfig"

In [ ]:
conn_prob_report = ByMtypeConnectionProbabilityAnalysis((CircuitAdapter(), CircuitModel(proj1_circuit)))

In [ ]:
syn_per_conn_report = ByMtypeSynapsesPerConnectionAnalysis((CircuitAdapter(), CircuitModel(proj1_circuit)))

In [ ]:
pathway_synapses_report = ByMtypePathwaySynapsesAnalysis((CircuitAdapter(), CircuitModel(proj1_circuit)))